## 任务一的思路和难点：
### 思路：
- 1、通过categories列来统计数量
- 2、如果属于cs(computer science)则做上1或者True的标记，方便统计计算
- 3、把标记存储在哪里呢，此时需要在data的最右侧添加新的一列，名字随意，我取的是'flag',默认填充为False
- 4、最后用sum()一统计就好了

### 难点：
- 发现一篇论文一般不单单只属于一个列别，比如'cs.AO math.AH'这就是同时属于两个种类，现在就需要使用正则表达式识别出'cs.AO',并在flag列标记为True

In [22]:
# id              : arXivID,用于访问论文
# submitter       : 论文提交者
# authors         : 论文作者
# title           : 论文标题
# comments        : 论文页数和图标等其它信息
# journal-ref     : 论文发表的期刊的信息
# doi             : 数字对象标识符
# report-no       : 报告编号
# categories      : 论文在arXiv系统的所属类别标签
# license         : 文章的许可证
# abstract        : 摘要
# versions        : 论文版本
# authors_parsed  : 作者信息

# BaseLine
# 任务1:论文数量统计（统计2019年全年，计算机各个方向论文的数量）
# 任务2:论文作者统计（统计所有论文作者出现频率Top10的姓名）
# 任务3:论文代码统计（统计所有论文类别下包含源代码论文的比例）
# 任务4:论文分类（利用已有数据建模，对新论文进行类别分类）
# 任务5:作者关联（对论文作者关系进行建模，统计最常出现的作者关系）

In [23]:
# 先用这几个库，之后不够再加
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
# 读取数据
# 一直没看明白为什么有的帖子说全部读取需要8G，我这里一下子就读取了啊

data = []
# 使用with语句优势：1.自动关闭文件句柄；2.自动显示（处理）文件读取数据异常
with open(r"D:\TianChi_DataSet\paper_set\arxiv-metadata-oai-2019.json", 'r') as f:
    for line in f:
        data.append(json.loads(line))
    data = pd.DataFrame(data)    # 将list变为dataframe格式，方便使用pandas进行分析
print(data.shape)   #显示数据大小

# 结果：2019年这个网站有17万篇论文

(170618, 14)


In [25]:
# 显示各个特征值
data.columns

Index(['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi',
       'report-no', 'categories', 'license', 'abstract', 'versions',
       'update_date', 'authors_parsed'],
      dtype='object')

In [26]:
# 任务一是统计计算机方向的论文数量
# arXiv那个网站加载不出来，我也懒得搭梯子，所以姑且认为 SC(Cumputer Science 计算机科学) 就是我们的目标类别

# 通过查看data中论文类别的列，发现一个难点，就是同一篇论文所属的种类有多个，所以我的第一个想法就是通过循环+正则表达式
# 的方式把多个类别当中的sc挑选出来，并且打标为True，或者1也行，然后用sum()函数直接统计数量就好啦
# 所以，让我们开始吧
data['categories'].head(10)

0    astro-ph
1     math.AT
2    astro-ph
3       gr-qc
4    astro-ph
5     nucl-ex
6    quant-ph
7     math.DG
8      hep-ex
9    astro-ph
Name: categories, dtype: object

In [27]:
# 首先，需要再data最右侧新加一列，用于存储之前提到的标记
data['flag'] = False
data.columns

Index(['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi',
       'report-no', 'categories', 'license', 'abstract', 'versions',
       'update_date', 'authors_parsed', 'flag'],
      dtype='object')

In [28]:
# 简单查看一下，确实添加成功了
data['flag'].head()

0    False
1    False
2    False
3    False
4    False
Name: flag, dtype: bool

In [29]:
# 现在开始用正则表达式选取属于sc的论文，并再flag列中标记为True
# 编写正则表达式处理函数
import re
def re_categories(s):
    str = s['categories']
    result = re.search('(cs\.[A-Z][A-Z])', str, flags=0)
    if isinstance(result, re.Match):
        return True
    else:
        return False
data['flag'] = data.apply(re_categories, axis=1)

In [30]:
# 简单看了一下100-200行之间的数据，看到115行，123行正确的变成了True，其它的还是False，目前看起来还是比较正确的
data.loc[100:200, ['categories', 'flag']].head(30)

,categories,flag
100,hep-ex,False
101,astro-ph,False
102,hep-ex,False
103,physics.plasm-ph,False
104,astro-ph,False
105,hep-ex,False
106,hep-ex,False
107,math.OA,False
108,hep-ex,False
109,hep-ex,False


In [31]:
# 最终的统计总和
num = data['flag'].sum()
print('2019年 计算机方向论文数量：', num)

2019年 计算机方向论文数量： 53888
